# Occupancy Grid PseudoCode

We first need to convert the actor_state_tensor to a more usable format, the information that we need should be stored into a dictionary like this:
```
{
    walls: [
        {pos_x: 0,
        pos_y: 0,
        width: 1,
        height: 10,
        theta: 0}, 
        ...
    ], 
    obstacles: [
        {pos_x: 0,
        pos_y: 0,
        width: 1,
        height: 1,
        theta: 0.32},
        ...
    ]
    robot: [
        {pos_x: 0, 
        pos_y: 0,
        width: 1, 
        height: 1, 
        theta: 2.5}
    ]
}
```

Given an object's `pos_x`, `pos_y`, `width`, `height`, the upper and lower bound space that the object occupies is:

x: [`pos_x` + `width`/2, `pos_x` - `width`/2] if orientation == 0 else  [`pos_x` + `height`/2, `pos_x` - `height`/2]

y: [`pos_y` + `height`/2, `pos_y` - `height`/2] if orientation == 0 else [`pos_y` + `width`/2, `pos_y` - `width`/2]

Let's assume that every room is the same dimension, and the occupancy grid represents a 10m x 10m space with a n x n grid

In this space we represent each cell with the following notation:
- 0: free space
- 1: wall
- 2: obstacle
- 3: robot

> Let's approach from the perspective that each object needs to be represented on the grid:

### Naive approach:
All cells that the x-y bound of the obstacle touches (with threshold > 25% of the grid?) will belong to this object;

Where there multiple objects negotiating for a single cell, we choose the object with the largest area present

should work if object size << grid size




```
class Occ_grid():
    

```